# Business Understanding
The project is designed to help host find characters of the most popular lodging and the factors affecting the price range. Five business interest questions will guide readers to learn about the competitive level of Seattle lodging.

### Question 1: When and where does travelers want to go

### Question 2: What does other host have for their lodging

### Question 3: What features affect lodging price the most?

# Data Understanding

### Gather data from csv files
There are three files provided from resouces: calender, listings and reviews

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import seaborn as sns
%matplotlib inline

#read data
calendar = pd.read_csv('calendar.csv')
listings = pd.read_csv('listings.csv')
reviews = pd.read_csv('reviews.csv')

## A look of the data
In Calendar file, there are 3181 lodging with records from 2016-01-04 to 2017-01-03

price would be NaN if available is f

### Calendar file

In [ ]:
calendar.shape

In [ ]:
calendar.head()

In [ ]:
calendar.groupby('date').count()['listing_id'].value_counts()

Calendar file is a date matters file recorded the listing data through out a year (365 days), from 2016-01-04 to 2017-01-02. Each date has listing status of 3818 lodgings, which means there are no missing listing. 

### Listings file

In [ ]:
listings.shape

In [ ]:
listings.head()

In [ ]:
listings.dtypes.value_counts()

In [ ]:
listings.select_dtypes('object').columns

In [ ]:
listings.select_dtypes(['float64','int64']).columns

There are 62 object variables and 30 numeric variables. 

The columns we are interested in are variables related to price and views. But the price-related variables are object type, meaning the data needs to be converted. 

At the same time, there are variables related to id and URL which should not be considered when doing analysis. 

### Reviews file

In [ ]:
reviews.head()

In [ ]:
reviews.shape

In [ ]:
#find the listing that has more than 1 reviews
multi_review = reviews.groupby('listing_id').filter(lambda x: x['comments'].count() > 1)
multi_review

In [ ]:
listing_count = len(set(reviews.listing_id))
listing_multi_review_count = len(set(multi_review.listing_id))
print('There are', listing_count, 'different lodging in the dataset,', 
     listing_multi_review_count, 'of them have more than 1 reviews')

In [ ]:
reviews.date.sort_values()

The reviews file gather comments from 2009-06-07 to 2016-01-03, before calendar file timeframe. 

# Data preparation

### *Reference: Defined Functions
Function that used for data cleaning

In [ ]:
def to_season(month: int):
    '''
    Convert month to season by [winter, spring, summer, fall] -> [0,1,2,3]
    
    input: 
            month: month in number format
    output: 
            season as in number, with (0, 1, 2, 3) as in (winter, spring, summer, fall)
    
    '''
    if month in [12,1,2]:
        return 0
    elif month in [3,4,5]:
        return 1
    elif month in [6,7,8]:
        return 2
    else: 
        return 3

In [ ]:
def divide_date_columns(dataframe, column_name, seperator = '-', date_sequence: list = ['year', 'month', 'day'], drop_org = False):
    '''
    Covert date columns to seperate columns with given seperator and sequence
    eg. orginal format as 1999-12-31 as [year, month, day] with seperator '-'
        
    input:    
            seperator: the charactor that seperate day, month, year in the original column
            date_sequence: a list with the order of year, month, day of original column
            drop_org: True/False, decide whether to drop the original columns
    output: 
            dataframe: new dataframe which is processed based on given parameters
    '''
    
    #test the whether column name is a list, if not, convert to list. 
    try:
        column_name = column_name.split()
    except: 
        pass
    
    #convert data with given column names
    for col in column_name: 
        for i, form in enumerate(date_sequence): 
            dataframe[col+'_'+form] = dataframe[col].apply(lambda date: date.split(seperator)[i])
        dataframe[col+'_season'] = dataframe[col+'_'+'month'].apply(lambda month: to_season(int(month)))
        if drop_org:
            dataframe.drop(col)
    return dataframe

In [ ]:
def convert_price(dataframe, column_name):
    '''
    convert price column to numeric value
    
    input: 
            dataframe: the dataframe that need to convert price columns to numeric value
            column_name: the name of price columns, can be list or a name of string
    output: 
            dataframe: new dataframe with target column converted
    '''
    
    #test the whether column name is a list, if not, convert to list. 
    try:
        column_name = column_name.split()
    except: 
        pass
        
    #convert data with given column names
    for col in column_name:
        dataframe[col] = dataframe[col].str.replace('[\$,]', '', regex=True).astype('float64').fillna(0)
    
    return dataframe

### Drop the column has large portion of missing value
Remove column that contain more than 80% NaN / Null

In [ ]:
listings = listings.drop(listings.columns[listings.isna().mean() > 0.80], axis = 1)

### Convert price column in listings and calendar

Remove symbol in price columns and convert to float. By the nature of price, we can take NaN value in price as 0, thus replace NaN value as 0. 

In [ ]:
#Calendar file
calendar = convert_price(calendar, 'price')

In [ ]:
#Lising file
col_price = ['price','weekly_price','monthly_price','security_deposit','cleaning_fee','extra_people']
listings = convert_price(listings, col_price)

In [ ]:
listings[col_price]

### Convert rate column to numeric

In [ ]:
col_rate = listings.columns[listings.columns.str.endswith('rate')]
for col in col_rate:
    listings[col] = listings[col].fillna(listings[col].mode()[0]).str.replace('%', '').astype('float64').apply(lambda row: row / 100)

### Fill NaN value in Review columns in listings file

In [ ]:
#find review column
col_review = listings.columns[listings.columns.str.startswith('review')]
listings[listings[col_review].isna().all(axis = 1)].shape[0]

### Clean categorical variable
Find the categorical variable, if numeric then convert to number; if there is only one category, then drop; if binary, replace with 0 and 1. 

In [ ]:
#make a list of object variable
col_obj = listings.select_dtypes('object').columns.tolist()

In [ ]:
#get rid of the column with only one category
for col in col_obj:
    if len(pd.unique(listings[col].str.lower())) == 1:
        listings = listings.drop(col, axis = 1)
        col_obj.remove(col)

In [ ]:
#replace binary column with f, t to 0, 1
col_binary = []
#Calendar file: replace available variable to 0 / 1
calendar['available'] = calendar['available'].apply(lambda x:1 if x=='t' else 0) 
calendar
for col in col_obj: #listings file
    if 'f' in pd.unique(listings[col].str.lower()):
        col_binary.append(col)
        listings[col] = listings[col].apply(lambda x:1 if x=='t' else 0)

In [ ]:
col_binary

### Seperate date column to three column as in year, month and day
in order to see the distribution of booking compare to month, obtain the month from date column

In [ ]:
calendar = divide_date_columns(calendar, 'date')

# Evaluate the Results

### Question 1: When and where does travelers want to go
Using the available rate, we can see the percentage of day that lodgingis not booked by traverler. The available rate will be calculted as percentage of available lodging and date within each zip code area and month. 

In [ ]:
calendar_available_permonth = calendar.groupby('date_month').mean()['available']
calendar_available_permonth.reset_index(inplace=True)
calendar_available_perlisting = calendar.groupby('listing_id').mean()['available']
calendar_available_perlisting = pd.DataFrame(calendar_available_perlisting).reset_index()

##### A line plot showing the trend of available rate over months

In [ ]:
plt.figure(figsize=(20, 10))
x = calendar_available_permonth['date_month']
y = calendar_available_permonth['available']
plt.plot(x,y,linewidth=3.0)
plt.xlabel('Month', )
plt.ylabel('Available rate')
plt.title('Month vs Available rate')
plt.axis([0, 12, 0.5, 0.8])
plt.grid(False)
plt.savefig('Available rate by month.png')
plt.show()

In [ ]:
calendar_available_perseason = calendar.groupby('date_season').mean()['available']
calendar_available_perseason = pd.DataFrame(calendar_available_perseason).reset_index()
calendar_available_perseason

In [ ]:
x = calendar_available_perseason.date_season
xi = list(range(len(x)))
y = calendar_available_perseason.available
plt.bar(xi, y, color ='maroon', width = 0.4)
plt.xticks(np.arange(0, 4, step=1))
plt.show()

### Question 2: What does other host have for their lodging

### Question 3: What features affect lodging price the most?

### In listings file, we are interest in price related and review related column
There are 90 variable describe the 3181 lodging, want to learn information like location, house size, lodging type and etc. 

In [ ]:
listings.isna().mean().sort_values()

#### Clean data, fill na and convert data type

### Find the distribution

##### The zipcode versus location review

In [ ]:
listings['zipcode'] = listings['zipcode'].replace('99\n98122', '98122') #modify content
listings.columns

In [ ]:
col_interest = ['review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'price', 'house size']

In [ ]:
# create the table of counts of the zipcode and average review
zipcode_counts = listings.zipcode.value_counts().sort_index()

# converting to df and assigning new names to the columns
df_zipcode_counts = pd.DataFrame(zipcode_counts)
df_zipcode_counts = df_zipcode_counts.reset_index()
df_zipcode_counts.columns = ['zipcode', 'counts'] # change column names
df_zipcode_counts['average_review'] = listings[['zipcode','review_scores_location']].groupby('zipcode').mean().reset_index()['review_scores_location']

In [ ]:
# 98118	154	9.000000
df_zipcode_counts = df_zipcode_counts.sort_values('average_review', ascending= False)

In [ ]:
df_zipcode_counts

#### hard to see the trend with line chart

In [ ]:
fig, ax1 = plt.subplots(figsize=(20, 8))

plt.figure(figsize=(20, 10))
x = df_zipcode_counts.sort_values('counts',ascending= False)['zipcode']
y = df_zipcode_counts.sort_values('counts',ascending= False)['counts']
y2 = df_zipcode_counts.sort_values('counts',ascending= False)['average_review']

color = 'tab:blue'
ax1.set_xlabel('zipcode')
ax1.set_ylabel('number of lodging', color = color)
ax1.plot(x, y, color = color)
ax1.tick_params(axis ='y', labelcolor = color)
  
ax2 = ax1.twinx()
  
color = 'tab:green'
ax2.set_ylabel('average_review', color = color)
ax2.plot(x, y2, color = color)
ax2.tick_params(axis ='y', labelcolor = color)
  
fig.suptitle('The distribution of lodging', fontweight ="regular", fontsize = 35)
plt.show()

#### availability per zipcode

In [ ]:
listings = listings.join(calendar_available_perlisting, lsuffix = 'id', rsuffix='listing_id')

In [ ]:
listings.groupby('zipcode').mean()['available'].to_csv('zipcode_available.csv')

#### house features: room number, bathrooms, beds, amenities etc. 

In [ ]:
#take the number of amenities as one of the house features 
listings['amenities_counts'] = listings['amenities'].str.replace('[{}]', '', regex=True).apply(lambda x: len(x.split(',')))

In [ ]:
amenities_list = listings['amenities'].str.replace('[{"}]', '', regex=True).apply(lambda x: x.split(','))
amenities = []
for row in amenities_list:
    amenities = list(set(amenities + row))
amenities.remove('')
# test = 0
# while test < 6:
#     print(listings['amenities'][test].split(','))
#     test += 1
# for row in listings['amenities']: 

In [ ]:
# import pprint
# pp = pprint.PrettyPrinter(width=80, compact=True)
# pp.pprint(amentities)
amenities.sort(key = len)
pd.DataFrame(amenities).to_excel('amenities.xlsx')

In [ ]:
listings.describe()['amenities_counts']

### Correlation with price

In [ ]:
col_id = listings.columns[listings.columns.str.endswith('id')]
listings.select_dtypes(['int64','float64']).drop(col_binary+list(col_review)+list(col_id),axis = 1).corr()['price'].sort_values(ascending = False)

In [ ]:
listings['price_per_person'] = listings['price']/listings['accommodates']

### Type of lodgings

In [ ]:
listings.groupby(listings['property_type']).mean()['price_per_person'].sort_values(ascending = False)

In [ ]:
listings.select_dtypes(['int64','float64']).drop(col_binary+list(col_review)+list(col_id),axis = 1).corr()['price_per_person'].sort_values(ascending = False)